<a href="https://colab.research.google.com/github/VitoZac96/Thesis_public/blob/main/Baseline_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline Model: CNN replicata dal paper: Time Series Classification from Scratch with Deep Neural Networks: A Strong Baseline

https://arxiv.org/abs/1611.06455 

Installare pacchetti per scaricare la timeseries dei prezzi dell'indice S&P500

In [ ]:
!pip install yfinance
!pip install yahoofinancials

In [4]:
import yfinance as yf
from yahoofinancials import YahooFinancials
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from keras.layers import Dropout

Funzione per creare gli input: le features che saranno usate nella CNN sono gli "n_steps" ritorni passati. 

In [5]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

Download the data, select only the closing price and compute the daily percentage change. 

In [6]:
spx = yf.download('^GSPC', 
                      start='2015-07-07', 
                      end='2021-03-01', 
                      progress=False)
spx = spx["Adj Close"].pct_change()[1:]

Creare x e y

In [7]:
x, y = split_sequence(spx,2)

Encoding per il classifier: se il ritorno di domani > 0: valore 1; 0 viceversa.

In [8]:
encoded = []
for i in y:
  if i>0:
    encoded.append(1)
  else:
    encoded.append(0)

y_target = pd.DataFrame(encoded).to_numpy()

Train test split e one hot encoding per il classifier

In [9]:
x_train, x_test, y_train, y_test_val = train_test_split(x, y_target, test_size=0.20, random_state=42)

y_train = keras.utils.to_categorical(y_train, 2)
y_test = keras.utils.to_categorical(y_test_val, 2)

Normalizzazione replicata dal paper

In [10]:
x_train_mean = x_train.mean()
x_train_std = x_train.std()
x_train = (x_train - x_train_mean)/(x_train_std)

In [11]:
x_test = (x_test - x_train_mean)/(x_train_std)    #### Perché non con media e std di se stesso?????

x_train = x_train.reshape(x_train.shape + (1,1,))
x_test = x_test.reshape(x_test.shape + (1,1,))

Modello replicato dal Paper

In [12]:
x = keras.layers.Input(x_train.shape[1:])
drop_out = Dropout(0.2)(x)
conv1 = keras.layers.Conv2D(128, 8, 1, padding='same')(x)
conv1 = keras.layers.BatchNormalization()(conv1)
conv1 = keras.layers.Activation('relu')(conv1)

drop_out = Dropout(0.1)(conv1)
conv2 = keras.layers.Conv2D(256, 5, 1, padding='same')(conv1)
conv2 = keras.layers.BatchNormalization()(conv2)
conv2 = keras.layers.Activation('relu')(conv2)

drop_out = Dropout(0.1)(conv2)
conv3 = keras.layers.Conv2D(128, 3, 1, padding='same')(conv2)
conv3 = keras.layers.BatchNormalization()(conv3)
conv3 = keras.layers.Activation('relu')(conv3)

full = keras.layers.GlobalAveragePooling2D()(conv3)
out = keras.layers.Dense(2, activation='sigmoid')(full)


model = keras.models.Model(inputs=x, outputs=out)
  
optimizer = keras.optimizers.Adam()
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
  
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = 'loss', factor=0.5, patience=50, min_lr=0.0001)  #cambiato da 1

hist = model.fit(x_train, y_train,  epochs=200, verbose=1, validation_data=(x_test, y_test), callbacks = [reduce_lr])
#batch_size=batch_size,

#Print the testing results which has the lowest training loss.
log = pd.DataFrame(hist.history)
print(log.loc[log['loss'].idxmin]['loss'], log.loc[log['loss'].idxmin]['val_accuracy'])

Epoch 1/200
36/36 [==============================] - 33s 21ms/step - loss: 0.7919 - accuracy: 0.5299 - val_loss: 0.6901 - val_accuracy: 0.6232
Epoch 2/200
36/36 [==============================] - 0s 6ms/step - loss: 0.6986 - accuracy: 0.5341 - val_loss: 0.6872 - val_accuracy: 0.6162
Epoch 3/200
36/36 [==============================] - 0s 6ms/step - loss: 0.6994 - accuracy: 0.5381 - val_loss: 0.6814 - val_accuracy: 0.6162
Epoch 4/200
36/36 [==============================] - 0s 6ms/step - loss: 0.6822 - accuracy: 0.5555 - val_loss: 0.6850 - val_accuracy: 0.6162
Epoch 5/200
36/36 [==============================] - 0s 6ms/step - loss: 0.6961 - accuracy: 0.5282 - val_loss: 0.6875 - val_accuracy: 0.6162
Epoch 6/200
36/36 [==============================] - 0s 6ms/step - loss: 0.6885 - accuracy: 0.5094 - val_loss: 0.6876 - val_accuracy: 0.6162
Epoch 7/200
36/36 [==============================] - 0s 6ms/step - loss: 0.6843 - accuracy: 0.5560 - val_loss: 0.6780 - val_accuracy: 0.6197
Epoch 8/200